# Basic imports

In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import DocArrayInMemorySearch

from operator import itemgetter

# Creating a model from Ollama

In [2]:
MODEL = 'llama3'

ollama_model = Ollama(model=MODEL)
ollama_embeddings = OllamaEmbeddings(model=MODEL)

In [3]:
ollama_model.invoke('Tell me a joke')

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(wait for it...)\n\nBecause it was two-tired!\n\nHope that made you smile! Do you want to hear another one?"

In [4]:
parser = StrOutputParser()

chain = ollama_model | parser
chain.invoke('Tell me a joke')

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(Wait for it...)\n\nBecause it was two-tired!\n\nHope that made you smile! Do you want to hear another one?"

# Loading a file to pass to the langchain

In [5]:
file = 'Resources/2024_Kosar_Computer science education in chatgpt era.pdf'

loader = PyPDFLoader(file)

pages = loader.load_and_split()

In [6]:
prompt_template = '''
You are a research assistant that is tasked with reviewing scientific papers. You are given different research questions that you have to answer by reading the papers. You will be given one paper at a time. You have to read the paper, create a summary of the paper, including its main contributions and drawbacks, and then answer the questions based on the content of the paper. Do not add information not present in the paper.

Context: {context}

Question: {question}
'''

In [7]:
prompt = PromptTemplate.from_template(prompt_template)
print(prompt.format(context='Here is some context', question='Here is a question'))


You are a research assistant that is tasked with reviewing scientific papers. You are given different research questions that you have to answer by reading the papers. You will be given one paper at a time. You have to read the paper, create a summary of the paper, including its main contributions and drawbacks, and then answer the questions based on the content of the paper. Do not add information not present in the paper.

Context: Here is some context

Question: Here is a question



# Create a chain

In [8]:
chain = prompt | ollama_model | parser

We need to use a vector store to store the contents of the pdf file to pass as context to the prompt.

In [9]:
vectore_store = DocArrayInMemorySearch.from_documents(pages, embedding = ollama_embeddings)

retriever = vectore_store.as_retriever()

e:\Programs\miniconda3\envs\research_ass\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [10]:
chain = (
    {
        'context': itemgetter('question') | retriever,
        'question': itemgetter('question')
    }
    | prompt
    | ollama_model
    | parser
)

# Run the chain on some questions

In [11]:
questions = [
    'What is the contribution of the paper?',
    'What are the implications for programming education?',
    'What data has been used in the paper?',
    'What are the advantages and disadvantages of the proposed approach?'
]

In [12]:
for question in questions:
    print(f'Question: {question}')
    print(f"Answer: {chain.invoke({'question': question})}")
    print('-------------------')

Question: What is the contribution of the paper?
Answer: The paper discusses the use of ChatGPT in a computer science education setting, specifically in a classroom where students were allowed to use it for certain purposes. The main contributions of the paper are:

1. An analysis of how students used ChatGPT in different ways, including code generation, optimization, comparison, and explanation.
2. A finding that students were able to understand the code generated by ChatGPT, with most participants rating it as "very understandable".
3. An observation that while some students did not follow instructions and used ChatGPT excessively, others were more responsible in their use of the technology.

The paper provides insights into how students interacted with ChatGPT and how educators can effectively integrate this technology into educational settings to promote learning.
-------------------
Question: What are the implications for programming education?
Answer: Based on the provided resear